In [19]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
#!pip install dask[dataframe]
# We have to do this as dask is bad at guessing the types of objects for some reason
# Also, if you are having trouble with running the dataframe and continuously get 
# Fatal Python error: GC object already tracked, try to update pandas
data_types = {
    'ID':  np.int64,
    'Case Number':  object,
    'Date': object,
    'Block': object,
    'IUCR': object,
    'Primary Type': object,
    'Description':  object,
    'Location Description': object,
    'Arrest':  bool,
    'Domestic': bool,
    'Beat': np.int64,
    'District': np.float64,
    'Ward': np.float64,
    'Community Area': np.float64,
    'FBI Code': object,
    'X Coordinate': np.float64,
    'Y Coordinate': np.float64,
    'Year': np.int64,
    'Updated On': object,
    'Latitude': np.float64,
    'Longitude': np.float64,
    'Location': object
}
data = dd.read_csv('crimes.csv', dtype=data_types)
data_subset = data.head(100)


In [20]:
from geopy.geocoders import Nominatim
# Need to install geopy
#!pip install geopy
geolocator = Nominatim()
location = geolocator.geocode(data_subset['Block'][0])
print location.latitude, location.longitude
print data_subset['Latitude'][0], data_subset['Longitude'][0]

41.8484243 -87.7209974
41.845400418 -87.720917872
